# 1) 
# Парсинг данных с [The_Witcher_3_food_and_drink]([asf](https://witcher-games.fandom.com/wiki/Category:The_Witcher_3_food_and_drink))

In [304]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import regex as re
# get all links to the food and drinks:
soup = BeautifulSoup(requests.get('https://witcher-games.fandom.com/wiki/Category:The_Witcher_3_food_and_drink').text)
Links = [i['href'] for i in soup.body.find_all("a", href=True) if i['href'][:5] == '/wiki'][12:-4]

# using regex find price, regen/sec and duration
# if there are no such values, then it is an empty page so we skip
BigList2 = []
for link in Links:
  try:
    name = link[6:]
    text = requests.get("https://witcher-games.fandom.com"+link).text
    match = re.search(r'buy.*?(\d+)', text, re.DOTALL)
    price = int(match.group(1))
    text = text[match.start():]
    match = re.search(r'(\d+)\s*Vitality regeneration', text, re.DOTALL)
    regen_s = int(match.group(1))
    text = text[match.start():]
    match = re.search(r'Duration.*?(\d+)', text, re.DOTALL)
    regen = int(match.group(1))*regen_s # full regen here
    BigList2.append([name, regen, price])
  except:
    print(link)

/wiki/Bottle_of_wine
/wiki/Golden_egg
/wiki/Jar_of_honey
/wiki/Potion_of_Restoration


# 2)
# Табличка с названиями, регеном/сек $\cdot$ Duration, ценой.
# Предположения условий задачи и анализ таблички

Заметим, что в данной фентезийной задаче не указаны наиболее важные аспекты, такие как:

a) Какова наша целевая аудитория покупателей: будут ли это игроки, которые стремятся максимизировать regen/price и будет ли их много\
b) Какова доступность средневековых первичных товаров: можем ли мы предполагать что их бесконечно?

От этих условий радикально меняется задача, и поэтому я сделаю наиболее естестввенные мне предположения, чтобы сузить анализ до адекватного масштаба:

a') Я рассмотрю два подхода: первый -- покупатели стремятся максимизировать соотношение regen/price и тем самым покупают лишь один продукт. второй ("компромисс") -- покупатели покупают все из топа 10 по соотношению regen/price. Для простоты мы предположим, что так как это фэнтэзи вселенная, то покупателей-игроков бесконечно много и мы можем мгновенно продать товар.\
b') Также для простоты мы предположим, что мы в бесконечном количестве можем закупать первичные товары по ценам, примерно равным найденным в интернете.

In [344]:
df = pd.DataFrame(BigList2, columns=['name', 'regen', 'price'])
df['regen/price'] = df['regen']/df['price']
df = df.sort_values(by='regen/price', ascending=False)
df = df.reset_index()
df = df.drop(columns=['index'])
df

,name,regen,price,regen/price
0,Horn_of_Plenty,200,0,inf
1,Potion_from_Tir_n%C3%A1_Lia,250,1,250.000000
2,Dumpling,250,1,250.000000
3,Butter,800,5,160.000000
4,Toffee,250,2,125.000000
5,Pepper,250,2,125.000000
6,Vinegar,250,4,62.500000
7,Raw_meat,250,4,62.500000
8,Blueberries,250,4,62.500000
9,Apple,250,5,50.000000


Поэтому, судя по табличке **при первом подходе** необходимо исследовать торговлю пельменями (Dumpling), так как рог изобилия и зелья мы попросту не можем изготовить

А при **втором подходе** в наших предположениях можно заметить, что некоторые товары мы можем мгновенно покупать/продавать и тогда\
pnl за день будет буквально бесконечным, но для хоть сколько-нибудь адекватной оценки дополнительно предположим что (в данном подходе) мы не можем покупать готовые для употребления продукты (то есть мы не можем купить оливки, а можем лишь косточки), тогда, очевидно, среди топ-10 самым выгодным вариантом окажется сырое мясо.

Сырое мясо наиболее выгодно потому, что первичный продукт - корова, которую для переработки в готовый продкут достаточно лишь (жестоко) убить с помощью раба и взять нужный кусок, в то время как для остальных вариантов необходимо время на созревание.

# 3)
# Подсчет PnL в год для (первый подход) Dumpling и (второй подход) Raw Meat
(стоимость любых вещей сильно варьируется по времени и географическому расположению, поэтому ограничимся средними оценками взятыми из каких-либо ресурсов)

## 1) Raw Meat: 

План: \
a) Купим рабов для работы, будем кормить их нашим продуктом (Raw Meat)\
b) Купим корову\
c) На каждую корову отделим $k$ рабов, чтобы они ее (жестоко) убивали и разделывали\
d) Мгновенно совершаем сделку по продаже 

Реализация:\
0) Будем считать, что серебряные монеты из игры - пенни (денье) [источник](https://en.wikipedia.org/wiki/%C2%A3sd)\
a) Будем считать, что раба можно купить за 2 лиры [источник](https://www.researchgate.net/figure/Average-Price-of-Female-and-Male-Slaves-in-Sale-Contracts-by-Decade_tbl5_232851490), 1 лира = 240 пенни (серебрянных монет) [источник](https://en.wikipedia.org/wiki/Venetian_lira)\
b) Корову купим по 10 шиллингов [источник](http://medieval.ucdavis.edu/120D/Money.html), это цена за хорошую корову, чтобы было меньше издержек, 1 шиллинг = 12 пенни\
c) время на убийство коровы ~ 20 сек, время на разделывание коровы ~ 1ч для 2 рабов [источник](https://www.quora.com/How-long-does-it-take-to-butcher-a-cow)\
d) Средний вес коровы 400кг [источник](http://fermagid.ru/korovy/180-skolko-vesit.html) а средний вес стейка на картинке [Raw Meat](https://witcher-games.fandom.com/wiki/Raw_meat) ~ 0.5 кг => Без учета трат на рабов, мы за час получаем $\approx $
$$4 \text{ пенни } \cdot 400 / 0.5 \text{ кусков } / 2 \text{ (делю на 2 так как не все мясо можно пустить на стейки)} - 10\cdot 12 \text{ пенни } = 1480 \text{ пенни в час}$$


Таким образом, имея $2N$ рабов, в год мы получаем $1480*24*365*N-$ \{траты на рабов\}.\
Рабам можно отдавать остатки коров и содержать в своем доме бесплатно. Так как они работают на износ, возможно придется покупать новые $2N$ рабов каждый год, таким образом имеем $1480*24*365*N-2*N*480 \approx N*10^6 \text{ пенни в год}$

## 2) Dumpling:

План: \
a) Купим рабов для работы, будем кормить их нашим продуктом (Dumpling)\
b) Купим зерно\
c) Купим мельницу и дадим рабам перемалывать зерно в муку\
d) Используем самое дешевое мясо (курицы)\
e) Заставим рабов делать Dumplings\
f) Мгновенно совершаем сделку по продаже

Реализация:\
a) Рабы по тем же ценам что и в Raw Meat, живут у нас дома.\
b) Зерно стоит $\approx 3$ шиллинга [см. стр. 81 в табличке](https://www.jstor.org/stable/pdf/2338591.pdf?refreqid=excelsior%3A22a57b93f2ef57144bfbd2d6b129b101&ab_segments=&origin=&initiator=&acceptTC=1) за 224кг
с) К сожалению, никаких данных о стоимости покупки мельницы не нашлось, но можем прикинуть, что простой замок по объему в ~ 10 раз больше мельницы,\
поэтому цена на мельницу не превысит 400/10 лир = 240*40 пенни [(стоимость замка можно взять тут)](https://castellogy.com/history/medieval-money)
d) 2 курицы (по ~ 1 кг) можно купить за 1 пенни [источник](http://medieval.ucdavis.edu/120D/Money.html)\
e) средневековые мельницы могут перерабатывать ~ 200кг зерна в день [источник](https://www.quora.com/How-would-a-windmill-be-used-to-help-in-the-creation-of-bread), производя 200\*0.75кг муки;\
на изготовку пельменей уходит меньше часа, средний вес пельмешки ~ 20 грамм\
1 кг муки + 1 кг мяса + 0.5 кг (взятой из озера вскипяченой бесплатной) воды = 2.5 кг пельменей. =>\
f) Итого, на 1 мельницу мы имеем в день:
$$(150+150+0.5*150) \text{ кг} / 0.02 \text{ в шт } * 1 \text{ пенни } - 150*0.5 \text{ пенни за 150 кг курицы} - 3*12*200/224\text{ пенни за 200кг зерна - \{расходы на рабов и мельницу\}}=$$
$$\approx 100 \text{ пенни в день}$$


Будем каждый год покупать новую мельницу и 10 рабов на содержание мельницы и готовку. 10 рабам дадим 5 куриц пропитания в день. =>\
Затраты составят $240*40 + 10*2*240 + 5/2 * 365 = 15312$ пенни в год. 

Таким образом имея N пачек \{мельница, 10 рабов\} наш чистый доход составит $\approx (100*365-15312)N=21188N$ пенни в год.

# 4)
# В рублях

Используя различные источники можно сделать вывод, что хоть сколько-нибудь адекватного дисконтирования цены сделать нельзя. (даже по индексу бургера...)\
Мне кажется интересным способ посчитать распределение состояния людей в то время и сейчас и взять взвешенное отношение состояния настоящего времени к прошлому как мультипликатор, но проблема в том, что таких данных не найти. Можно более грубо взять среднего по состоянию и среднего богатого (учитель и барон) [6 фунтов](https://thehistoryofengland.co.uk/resource/medieval-prices-and-wages/) и [350 фунтов в год](https://www.quora.com/How-much-money-did-medieval-lords-make) соответственно; против средней заработной платы в европе [17748 фунтов в год](https://en.wikipedia.org/wiki/List_of_European_countries_by_average_wage) и опытного квантового аналитика в UK [156000 фунтов в год](https://www.quora.com/What-is-the-average-salary-of-a-Quant-in-US-UK-and-Europe-as-a-beginner-and-how-does-it-vary-with-experience). Таким образом соотношения получаются ~2800 и ~450, то есть в этом промежутке стоит искать мультипликатор.


В статье [источник](https://castellogy.com/history/medieval-money) собраны наиболее популярные способы дисконтировать средневековую цену на современное время. Одним из способов наиболее естественно это сделать был бы взять все известные способы взвешенной суммой с весами пропорциональными актуальности к нашей задаче. Поэтому, вместо того чтобы сопоставлять распределение бедных/богатых людей тогда и сейчас, лучше воспользуемся "Индексом потребительских цен" [источник](https://castellogy.com/history/medieval-money), который учитывает цены и проторгованный объем продовольственных товаров (что наиболее близко нашей задаче). \
Данный индекс предлагает соотношение £1 в 1270 стоил бы сегодня £940 = 116*940 руб

Таким образом, для **первого подхода**, Dumplings дадут прирост на $N*21188/240 * 116*940 \approx 10^6N$ рублей/год, в то время как\
для **второго подхода**, Raw Meat даcт прирост в $N*10^6/240 * 116*940 \approx 4*10^8N$ рублей/год